In [1]:
from pydub import AudioSegment
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

ModuleNotFoundError: No module named 'keras'

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

In [3]:
data = AudioSegment.from_mp3("./alegrias.mp3")
data_np = np.array(data.get_array_of_samples())

In [4]:
data_np.shape

(571392,)

In [5]:
data_resampled = data.set_frame_rate(16000)
data_resampled_np = np.array(data_resampled.get_array_of_samples())

In [6]:
data_resampled_np.shape

(207308,)

In [7]:
data_resampled_np.shape[0] / 16000

12.95675

In [8]:
model = hub.load('https://tfhub.dev/google/vggish/1')

2021-10-16 09:06:42.469273: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model(data_resampled_np)

2021-10-16 09:06:43.638802: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-16 09:06:43.664891: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3491910000 Hz


<tf.Tensor: shape=(13, 128), dtype=float32, numpy=
array([[-0.9781215 , -0.23949593,  0.3278644 , ..., -0.70464456,
         0.15656663,  0.22936888],
       [-0.97239923, -0.26644686,  0.22860575, ..., -0.78322506,
         0.28337187,  0.03754485],
       [-1.1073772 , -0.25849837,  0.39797044, ..., -1.0069251 ,
         0.21025334, -0.02065074],
       ...,
       [-1.0072911 , -0.38478103,  0.23967987, ..., -0.6641028 ,
         0.14501332,  0.10112442],
       [-1.0529892 , -0.29740486,  0.30597043, ..., -0.7344618 ,
         0.1593184 ,  0.05672863],
       [-1.2255527 , -0.30298266,  0.4057959 , ..., -0.88262177,
         0.26835507, -0.03802034]], dtype=float32)>

In [2]:
df = pd.read_csv("../../assets/dataset/encoded_audio.csv", index_col=0)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,label
0,-0.269944,-0.054643,-0.137082,0.295747,-0.050220,-0.071286,-0.042822,-0.105079,-0.175809,-0.056513,...,-0.033017,-0.118352,-0.031682,-0.033670,-0.104144,0.205717,-0.162219,0.228777,0.177694,alegrias
1,-1.907480,-0.456176,0.862031,-0.410450,-0.904798,-1.543526,-0.814925,-0.409570,-3.509268,-1.167431,...,-0.421366,-0.711560,-0.850614,-1.387193,-0.268451,0.204029,-1.517510,-0.092862,-0.027284,alegrias
2,-0.845307,-0.194419,0.541575,-0.583690,-0.457475,-1.116324,0.407395,0.185402,-2.120158,-0.435578,...,-0.353207,-0.035378,-0.216617,-0.963617,-0.108042,0.036861,-0.529865,0.221667,0.301169,alegrias
3,-0.877723,-0.226133,0.661279,-0.495370,-0.389753,-1.189388,0.272059,0.287067,-2.324146,-0.359669,...,-0.327581,-0.083167,-0.299293,-0.866714,-0.252097,0.010195,-0.532761,0.142866,0.341006,alegrias
4,-1.427469,-0.349206,0.490714,-0.783012,-0.795806,-1.423361,0.075089,-0.029320,-2.920721,-0.679675,...,-0.328377,-0.171545,-0.302092,-1.034035,-0.440447,0.018078,-0.943624,0.134962,0.233029,alegrias


In [3]:
y = df["label"]
X = df.drop(columns=["label"])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
model = SGDClassifier(eta0=0.01, n_jobs=-1, random_state=42)

params = {
    "loss": [ "hinge", "log", "modified_huber", "squared_hinge", "perceptron", "huber", "epsilon_insensitive"],
    "penalty": ["l2", "l1", "elasticnet"],
    "alpha": np.linspace(0.0001, 0.1, 50),
    "l1_ratio": np.linspace(0.1, 1, 20),
    "learning_rate": ["constant", "optimal", "invscaling", "adaptive"]
}

scorers = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score, average="micro"),
    "f1": make_scorer(f1_score, average="micro")
}

clf = RandomizedSearchCV(model, params, refit=False, random_state=42, n_jobs=-1, verbose=2, n_iter=50, scoring=scorers)
clf.fit(X, y)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END alpha=0.018448979591836733, l1_ratio=0.4789473684210527, learning_rate=constant, loss=log, penalty=l2; total time=   3.0s
[CV] END alpha=0.018448979591836733, l1_ratio=0.4789473684210527, learning_rate=constant, loss=log, penalty=l2; total time=   4.7s
[CV] END alpha=0.018448979591836733, l1_ratio=0.4789473684210527, learning_rate=constant, loss=log, penalty=l2; total time=   1.5s
[CV] END alpha=0.018448979591836733, l1_ratio=0.4789473684210527, learning_rate=constant, loss=log, penalty=l2; total time=   2.5s
[CV] END alpha=0.018448979591836733, l1_ratio=0.4789473684210527, learning_rate=constant, loss=log, penalty=l2; total time=   1.2s
[CV] END alpha=0.0001, l1_ratio=0.5736842105263158, learning_rate=constant, loss=epsilon_insensitive, penalty=elasticnet; total time=   3.8s
[CV] END alpha=0.0001, l1_ratio=0.5736842105263158, learning_rate=constant, loss=epsilon_insensitive, penalty=elasticnet; total time=   3.8s
[

/home/ale/miniconda3/envs/conda-tf/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


[CV] END alpha=0.053108163265306124, l1_ratio=0.33684210526315794, learning_rate=optimal, loss=squared_hinge, penalty=l1; total time= 1.9min


/home/ale/miniconda3/envs/conda-tf/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


[CV] END alpha=0.053108163265306124, l1_ratio=0.33684210526315794, learning_rate=optimal, loss=squared_hinge, penalty=l1; total time= 1.9min


...

/home/ale/miniconda3/envs/conda-tf/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


[CV] END alpha=0.06330204081632654, l1_ratio=1.0, learning_rate=optimal, loss=squared_hinge, penalty=l1; total time= 1.8min
[CV] END alpha=0.0673795918367347, l1_ratio=0.9052631578947369, learning_rate=constant, loss=epsilon_insensitive, penalty=l2; total time=   1.3s
[CV] END alpha=0.08165102040816327, l1_ratio=0.19473684210526315, learning_rate=adaptive, loss=log, penalty=l1; total time=   5.3s
[CV] END alpha=0.0673795918367347, l1_ratio=0.9052631578947369, learning_rate=constant, loss=epsilon_insensitive, penalty=l2; total time=   1.0s
[CV] END alpha=0.08165102040816327, l1_ratio=0.19473684210526315, learning_rate=adaptive, loss=log, penalty=l1; total time=   5.8s
[CV] END alpha=0.0673795918367347, l1_ratio=0.9052631578947369, learning_rate=constant, loss=epsilon_insensitive, penalty=l2; total time=   0.9s
[CV] END alpha=0.0673795918367347, l1_ratio=0.9052631578947369, learning_rate=constant, loss=epsilon_insensitive, penalty=l2; total time=   1.3s
[CV] END alpha=0.0673795918367347,

/home/ale/miniconda3/envs/conda-tf/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


[CV] END alpha=0.06330204081632654, l1_ratio=1.0, learning_rate=optimal, loss=squared_hinge, penalty=l1; total time= 1.6min
[CV] END alpha=0.051069387755102044, l1_ratio=0.6210526315789474, learning_rate=adaptive, loss=perceptron, penalty=elasticnet; total time=   4.9s
[CV] END alpha=0.051069387755102044, l1_ratio=0.6210526315789474, learning_rate=adaptive, loss=perceptron, penalty=elasticnet; total time=   5.1s
[CV] END alpha=0.051069387755102044, l1_ratio=0.6210526315789474, learning_rate=adaptive, loss=perceptron, penalty=elasticnet; total time=   4.9s
[CV] END alpha=0.051069387755102044, l1_ratio=0.6210526315789474, learning_rate=adaptive, loss=perceptron, penalty=elasticnet; total time=   4.8s
[CV] END alpha=0.09184489795918369, l1_ratio=0.6210526315789474, learning_rate=optimal, loss=modified_huber, penalty=l1; total time=   1.5s
[CV] END alpha=0.09184489795918369, l1_ratio=0.6210526315789474, learning_rate=optimal, loss=modified_huber, penalty=l1; total time=   1.5s
[CV] END alp

RandomizedSearchCV(estimator=SGDClassifier(eta0=0.01, n_jobs=-1,
                                           random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': array([0.0001    , 0.00213878, 0.00417755, 0.00621633, 0.0082551 ,
       0.01029388, 0.01233265, 0.01437143, 0.0164102 , 0.01844898,
       0.02048776, 0.02252653, 0.02456531, 0.02660408, 0.02864286,
       0.03068163, 0.03272041, 0.03475918, 0.03679796, 0.03883673,...
                                        'learning_rate': ['constant', 'optimal',
                                                          'invscaling',
                                                          'adaptive'],
                                        'loss': ['hinge', 'log',
                                                 'modified_huber',
                                                 'squared_hinge', 'perceptron',
                                                 'huber',
                   

In [23]:
pd.DataFrame(clf.cv_results_).to_csv("sgd_results.csv")

In [30]:
model_forest = RandomForestClassifier(n_jobs=-1, random_state=42, verbose=2)

model_forest.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.8s


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   11.0s finished


RandomForestClassifier(n_jobs=-1, random_state=42, verbose=2)

In [34]:
y_pred = model_forest.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred, average="micro"))
print(precision_score(y_test, y_pred, average="micro"))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s


0.7017819222601791
0.7017819222601791
0.7017819222601791


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


In [36]:
model_forest = RandomForestClassifier(n_jobs=-1, random_state=42)

params = {
    "n_estimators": np.linspace(50, 600, 100, dtype=int),
    "criterion": ["gini", "entropy"],
    "max_depth": [None, *np.linspace(20, 150, 100, dtype=int)],
    "min_samples_split": np.linspace(2, 100, 10, dtype=int),
    "min_samples_leaf": np.linspace(1, 50, 10, dtype=int),
    "max_features": ["auto", "sqrt", "log2"],
}

scorers = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score, average="micro"),
    "f1": make_scorer(f1_score, average="micro")
}

clf_forest = RandomizedSearchCV(model_forest, params, refit=False, random_state=42, n_jobs=-1, verbose=2, n_iter=50, scoring=scorers)
clf_forest.fit(X, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time= 1.6min
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time= 1.6min
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time= 1.6min
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time= 1.7min
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time= 1.6min
[CV] END criterion=entropy, max_depth=73, max_features=sqrt, min_samples_leaf=22, min_samples_split=56, n_estimators=450; total time= 4.2min
[CV] END criterion=entropy, max_depth=73, max_features=sqrt, min_samples_leaf=22, min_samples_split=56, n_e

RandomizedSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 20, 21, 22, 23, 25,
                                                      26, 27, 29, 30, 31, 33,
                                                      34, 35, 37, 38, 39, 41,
                                                      42, 43, 44, 46, 47, 48,
                                                      50, 51, 52, 54, 55, 56, ...],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': array([ 1,  6, 11, 17, 22, 28, 33, 39, 44,...
       338, 344, 350, 355, 361, 366, 372, 377, 383, 388, 394, 400, 405,
       411, 416, 422, 427, 433, 438, 444, 450, 455, 461, 466, 472, 477,
       483, 488, 

In [37]:
pd.DataFrame(clf_forest.cv_results_).to_csv("forest_results.csv")

In [40]:
model_extra = ExtraTreesClassifier(n_jobs=-1, random_state=42)

params = {
    "n_estimators": np.linspace(50, 600, 100, dtype=int),
    "criterion": ["gini", "entropy"],
    "max_depth": [None, *np.linspace(20, 150, 100, dtype=int)],
    "min_samples_split": np.linspace(2, 100, 10, dtype=int),
    "min_samples_leaf": np.linspace(1, 50, 10, dtype=int),
    "max_features": ["auto", "sqrt", "log2"],
}

scorers = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score, average="micro"),
    "f1": make_scorer(f1_score, average="micro")
}

clf_extra = RandomizedSearchCV(model_extra, params, refit=False, random_state=42, n_jobs=-1, verbose=2, n_iter=50, scoring=scorers)
clf_extra.fit(X, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time=  25.4s
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time=  25.6s
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time=  25.7s
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time=  25.8s
[CV] END criterion=gini, max_depth=92, max_features=sqrt, min_samples_leaf=11, min_samples_split=78, n_estimators=288; total time=  24.0s
[CV] END criterion=entropy, max_depth=73, max_features=sqrt, min_samples_leaf=22, min_samples_split=56, n_estimators=450; total time=  37.2s
[CV] END criterion=entropy, max_depth=73, max_features=sqrt, min_samples_leaf=22, min_samples_split=56, n_e

RandomizedSearchCV(estimator=ExtraTreesClassifier(n_jobs=-1, random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 20, 21, 22, 23, 25,
                                                      26, 27, 29, 30, 31, 33,
                                                      34, 35, 37, 38, 39, 41,
                                                      42, 43, 44, 46, 47, 48,
                                                      50, 51, 52, 54, 55, 56, ...],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': array([ 1,  6, 11, 17, 22, 28, 33, 39, 44, 50...
       338, 344, 350, 355, 361, 366, 372, 377, 383, 388, 394, 400, 405,
       411, 416, 422, 427, 433, 438, 444, 450, 455, 461, 466, 472, 477,
       483, 488,

In [ ]:
pd.DataFrame(clf_extra.cv_results_).to_csv("extra_results.csv")

In [9]:
import json

In [19]:
pd.read_csv("./sgd_results.csv").sort_values(by=["rank_test_precision"]).loc[0, ["mean_test_accuracy", "mean_test_precision", "mean_test_f1"]]

mean_test_accuracy     0.454507
mean_test_precision    0.454507
mean_test_f1           0.454507
Name: 0, dtype: object

In [16]:
pd.read_csv("./sgd_results.csv").sort_values(by=["rank_test_precision"]).iloc[0]["params"]

"{'penalty': 'l1', 'loss': 'squared_hinge', 'learning_rate': 'optimal', 'l1_ratio': 0.33684210526315794, 'alpha': 0.053108163265306124}"

In [20]:
pd.read_csv("./forest_results.csv").sort_values(by=["rank_test_precision"]).loc[0, ["mean_test_accuracy", "mean_test_precision", "mean_test_f1"]]

mean_test_accuracy     0.608369
mean_test_precision    0.608369
mean_test_f1           0.608369
Name: 0, dtype: object

In [13]:
pd.read_csv("./forest_results.csv").sort_values(by=["rank_test_precision"]).iloc[0]["params"]

"{'n_estimators': 211, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 72, 'criterion': 'entropy'}"

In [21]:
pd.read_csv("./extra_results.csv").sort_values(by=["rank_test_precision"]).loc[0, ["mean_test_accuracy", "mean_test_precision", "mean_test_f1"]]

mean_test_accuracy     0.597936
mean_test_precision    0.597936
mean_test_f1           0.597936
Name: 0, dtype: object

In [14]:
pd.read_csv("./extra_results.csv").sort_values(by=["rank_test_precision"]).iloc[0]["params"]

"{'n_estimators': 211, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 72, 'criterion': 'entropy'}"

In [4]:
model_forest = RandomForestClassifier(
    n_estimators=211,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='auto',
    max_depth=72,
    criterion='entropy',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

model_forest.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 211building tree 2 of 211
building tree 3 of 211
building tree 4 of 211

building tree 5 of 211
building tree 6 of 211
building tree 7 of 211
building tree 8 of 211
building tree 9 of 211
building tree 10 of 211
building tree 11 of 211
building tree 12 of 211
building tree 13 of 211
building tree 14 of 211
building tree 15 of 211
building tree 16 of 211
building tree 17 of 211
building tree 18 of 211
building tree 19 of 211
building tree 20 of 211
building tree 21 of 211
building tree 22 of 211
building tree 23 of 211
building tree 24 of 211
building tree 25 of 211
building tree 26 of 211
building tree 27 of 211
building tree 28 of 211
building tree 29 of 211
building tree 30 of 211
building tree 31 of 211
building tree 32 of 211
building tree 33 of 211
building tree 34 of 211
building tree 35 of 211
building tree 36 of 211
building tree 37 of 211
building tree 38 of 211


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.1s


building tree 39 of 211
building tree 40 of 211
building tree 41 of 211
building tree 42 of 211
building tree 43 of 211
building tree 44 of 211
building tree 45 of 211
building tree 46 of 211
building tree 47 of 211
building tree 48 of 211
building tree 49 of 211
building tree 50 of 211
building tree 51 of 211
building tree 52 of 211
building tree 53 of 211
building tree 54 of 211
building tree 55 of 211
building tree 56 of 211
building tree 57 of 211
building tree 58 of 211
building tree 59 of 211
building tree 60 of 211
building tree 61 of 211
building tree 62 of 211
building tree 63 of 211
building tree 64 of 211
building tree 65 of 211
building tree 66 of 211
building tree 67 of 211
building tree 68 of 211
building tree 69 of 211
building tree 70 of 211
building tree 71 of 211
building tree 72 of 211
building tree 73 of 211
building tree 74 of 211
building tree 75 of 211
building tree 76 of 211
building tree 77 of 211
building tree 78 of 211
building tree 79 of 211
building tree 80

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   39.5s


building tree 160 of 211
building tree 161 of 211
building tree 162 of 211
building tree 163 of 211
building tree 164 of 211
building tree 165 of 211
building tree 166 of 211
building tree 167 of 211
building tree 168 of 211
building tree 169 of 211
building tree 170 of 211
building tree 171 of 211
building tree 172 of 211
building tree 173 of 211
building tree 174 of 211
building tree 175 of 211
building tree 176 of 211
building tree 177 of 211
building tree 178 of 211
building tree 179 of 211
building tree 180 of 211
building tree 181 of 211
building tree 182 of 211
building tree 183 of 211
building tree 184 of 211
building tree 185 of 211
building tree 186 of 211
building tree 187 of 211
building tree 188 of 211
building tree 189 of 211
building tree 190 of 211
building tree 191 of 211
building tree 192 of 211
building tree 193 of 211
building tree 194 of 211
building tree 195 of 211
building tree 196 of 211
building tree 197 of 211
building tree 198 of 211
building tree 199 of 211


[Parallel(n_jobs=-1)]: Done 211 out of 211 | elapsed:   53.9s finished


RandomForestClassifier(criterion='entropy', max_depth=72, n_estimators=211,
                       n_jobs=-1, random_state=42, verbose=2)

In [12]:
joblib.dump(model_forest, "random_forest.joblib", compress=3)

['random_forest.joblib']